In [1]:
from __future__ import division
import os
import sys
import time
import glob
import logging
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.utils
from torch.utils.data import DataLoader
from torch.utils.data.dataloader import default_collate
from tensorboardX import SummaryWriter
import numpy as np
import matplotlib
matplotlib.use('Agg')
from thop import profile
from search.config_search import config
from search.crowd_dataloader import Crowd
from tools.utils.init_func import init_weight
from search.loss import Criterion
from search.architect import Architect
from tools.utils.darts_utils import create_exp_dir, save, plot_op, plot_path_width, objective_acc_lat
from search.supernet import Network_Multi_Path as Network

def train_collate(batch):
    transposed_batch = list(zip(*batch))
    images = torch.stack(transposed_batch[0], 0)
    points = transposed_batch[1]  # the number of points is not fixed, keep it as a list of tensor
    targets = transposed_batch[2]
    st_sizes = torch.FloatTensor(transposed_batch[3])
    return images, points, targets, st_sizes

device = torch.device("cpu")
criterion = Criterion(config.sigma, config.crop_size, config.downsample_ratio, config.background_ratio, config.use_background, device)
model = Network(config.layers, criterion, Fch=config.Fch, width_mult_list=config.width_mult_list, prun_modes=config.prun_modes, stem_head_width=config.stem_head_width)    

In [3]:
x = torch.randn(1, 3, 128, 512).requires_grad_(True)
y = model(x)

In [4]:
y

(tensor([[[[0.1751, 0.2451, 0.2062,  ..., 0.3210, 0.5065, 0.2820],
           [0.4948, 0.0515, 0.1126,  ..., 0.2421, 0.6521, 0.1064],
           [0.3545, 0.5859, 0.0124,  ..., 0.1415, 0.7260, 0.1957],
           ...,
           [1.4601, 0.8843, 0.0924,  ..., 0.0429, 0.3224, 0.2905],
           [1.2516, 1.6696, 1.7303,  ..., 0.0270, 0.4886, 0.1144],
           [1.0536, 2.1934, 3.1239,  ..., 0.7406, 0.7889, 0.3120]]]],
        grad_fn=<AbsBackward>),
 tensor([[[[0.0776, 0.0194, 0.2538,  ..., 0.2324, 0.1618, 0.4200],
           [0.1094, 0.1024, 0.4484,  ..., 0.1333, 0.1354, 0.6508],
           [0.0551, 0.1543, 0.1607,  ..., 0.6886, 0.6207, 0.2331],
           ...,
           [0.3602, 0.0858, 0.5713,  ..., 0.0220, 0.2591, 0.1559],
           [0.1153, 0.9001, 0.2032,  ..., 0.3145, 0.2988, 0.1616],
           [0.6379, 0.7104, 1.2097,  ..., 0.2826, 0.2671, 0.3041]]]],
        grad_fn=<AbsBackward>),
 tensor([[[[0.1510, 0.1472, 0.2107,  ..., 0.9774, 0.6907, 1.2334],
           [0.3848, 0.5845,

In [2]:
from torchviz import make_dot
x = torch.randn(1, 3, 512, 512).requires_grad_(True)
y = model(x)
vis_graph = make_dot(y, params=dict(list(model.named_parameters()) + [('x', x)]))
vis_graph.view()

ExecutableNotFound: failed to execute ['dot', '-Tpdf', '-O', 'Digraph.gv'], make sure the Graphviz executables are on your systems' PATH

In [ ]:
pip uninstall torchviz